# Εισαγωγή των απαραίτητων βιβλιοθηκών

In [ ]:
# ============================================================
# 1) Εγκατάσταση και εισαγωγή βιβλιοθηκών
# ============================================================

# Εισαγωγή απαραίτητων βιβλιοθηκών
import nltk
import re
import json
import math
import numpy as np
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# Αρχικοποίηση NLTK

In [ ]:
# Λήψη απαραίτητων δεδομένων NLTK (tokenizer, stopwords κ.λπ.)
nltk.download('punkt', force=True)
nltk.download('stopwords', force=True)
nltk.download('wordnet', force=True)

# Αρχικοποίηση NLTK components
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

print("Setup complete.")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Setup complete.


# Φόρτωση αρχείων

Σε αυτή την ενότητα ορίζουμε συναρτήσεις για να φορτώνουμε τα αρχεία JSON (με άρθρα/τίτλους) και το αρχείο CISI.QRY (με queries), έπειτα τα καλούμε για να φορτώσουμε τα δεδομένα μας.

Σημειώστε ότι αναμένεται να υπάρχουν τα αρχεία:

./wikipedia_articles.json (τα δεδομένα εισόδου που συλλέκτηκαν απο το web crawling στην βικιπαίδεια)

./processed_CISI_articles.json (περιέχει μια λίστα άρθρων, όπου το κάθε άρθρο έχει id και μια λίστα από tokens)

./CISI_articles.json (το πρωτότυπο αρχείο με πλήρη δεδομένα ή τουλάχιστον τα id και title)

./CISI.QRY (τα queries σε φορμά .I <id>, .W, κείμενο ...)

In [ ]:
# ============================================================
# 2) Φόρτωση άρθρων, τίτλων, ερωτημάτων
# ============================================================

def load_articles(json_file):
    """
    Φορτώνει από JSON αρχείο μια λίστα από άρθρα,
    όπου κάθε άρθρο είναι λεξικό με κλειδιά:
      "id", "tokens", πιθανώς και άλλα πεδία.
    """
    try:
        with open(json_file, 'r', encoding='utf-8') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Σφάλμα: Το αρχείο '{json_file}' δεν βρέθηκε.")
        return []
    except json.JSONDecodeError:
        print(f"Σφάλμα: Μη έγκυρο JSON στο '{json_file}'.")
        return []

def load_titles(json_file):
    """
    Φορτώνει από JSON αρχείο μια λίστα articles και φτιάχνει
    ένα λεξικό {article_id: article_title}.
    """
    try:
        with open(json_file, 'r', encoding='utf-8') as file:
            articles = json.load(file)
        return {article['id']: article['title'] for article in articles}
    except FileNotFoundError:
        print(f"Σφάλμα: Το αρχείο '{json_file}' δεν βρέθηκε.")
        return {}
    except json.JSONDecodeError:
        print(f"Σφάλμα: Μη έγκυρο JSON στο '{json_file}'.")
        return {}

def load_queries(file_path):
    """
    Διαβάζει από αρχείο (π.χ. CISI.QRY) τα queries σε φορμά:
      .I <query_id>
      .W
      <query text...>
    και επιστρέφει ένα λεξικό { query_id: query_text }.
    """
    queries = {}
    current_id = None
    query_text = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if line.startswith('.I'):
                if current_id is not None:
                    queries[current_id] = " ".join(query_text).strip()
                current_id = int(line.split()[1])
                query_text = []
            elif line.startswith('.W'):
                continue
            else:
                query_text.append(line)
        if current_id is not None:
            queries[current_id] = " ".join(query_text).strip()
    return queries

# -----------------------------------------------------------
# Τώρα καλούμε τις συναρτήσεις για να φορτώσουμε τα αρχεία μας
# -----------------------------------------------------------

articles_file = './processed_CISI_articles.json'
titles_file   = './CISI_articles.json'
queries_file  = './CISI.QRY'

articles = load_articles(articles_file)
title_mapping = load_titles(titles_file)
queries = load_queries(queries_file)

print(f"Φορτώθηκαν {len(articles)} άρθρα.")
print(f"Φορτώθηκαν {len(title_mapping)} τίτλοι άρθρων.")
print(f"Φορτώθηκαν {len(queries)} queries.")


Φορτώθηκαν 1460 άρθρα.
Φορτώθηκαν 1460 τίτλοι άρθρων.
Φορτώθηκαν 57 queries.


Σε αυτή την ενότητα δημιουργούμε μια συνάρτηση που κάνει preprocessing (επεξεργασία κειμένου) — αφαιρεί μη αλφαβητικούς χαρακτήρες, κάνει tokenize, αφαιρεί stopwords, εφαρμόζει stemming και lemmatization

In [ ]:
# ============================================================
# 3) Συνάρτηση προεπεξεργασίας κειμένου
# ============================================================

nltk.download('punkt_tab')

def process_query(text):
    """
    Βήματα:
      1) Αφαίρεση μη αλφαβητικών χαρακτήρων
      2) Tokenize
      3) Lowercase
      4) Αφαίρεση stopwords
      5) Stemming
      6) Lemmatization
    """
    cleaned_text = re.sub(r'[^A-Za-z\s]', '', text)
    tokens = word_tokenize(cleaned_text.lower())
    filtered_tokens = [w for w in tokens if w not in stop_words]
    stemmed_tokens = [stemmer.stem(w) for w in filtered_tokens]
    lemmatized_tokens = [lemmatizer.lemmatize(w) for w in stemmed_tokens]
    return lemmatized_tokens

# -----------------------------------------------------------
# Δοκιμή της συνάρτησης
# -----------------------------------------------------------
sample_text = "Information retrieval is one of the most important subjects!"
processed_tokens = process_query(sample_text)

print("Original text: ", sample_text)
print("Processed tokens: ", processed_tokens)


Original text:  Information retrieval is one of the most important subjects!
Processed tokens:  ['inform', 'retriev', 'one', 'import', 'subject']


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


# Inverted Index Implementation
Εδώ φτιάχνουμε το inverted index, που είναι ένα λεξικό:
token -> [doc_id1, doc_id2, ...],
για να γίνει συσχέτιση των όρων και σε ποια documents ανήκουν

In [ ]:
# ============================================================
# 4) Υλοποίηση Inverted Index για Boolean Search
# ============================================================

def buildInvertedIndex(articles):
    """
    Επιστρέφει ένα λεξικό { token: [doc_id1, doc_id2, ...] }.
    """
    inverted_index = defaultdict(list)
    for article in articles:
        for token in set(article["tokens"]):
            inverted_index[token].append(article["id"])
    return inverted_index

def searchIndex(term, inverted_index):
    """
    Επιστρέφει ένα σύνολο (set) με τα doc_ids για τον όρο 'term'.
    Αν δεν υπάρχει ο όρος, επιστρέφει κενό set.
    """
    return set(inverted_index.get(term, []))

# -----------------------------------------------------------
# Δημιουργία του inverted_index
# -----------------------------------------------------------
inverted_index = buildInvertedIndex(articles)
print("Inverted index built.")

# -----------------------------------------------------------
# Δοκιμή στο πρώτο token του sample_text
# -----------------------------------------------------------
if processed_tokens:
    sample_term = processed_tokens[0]
    matching_docs = searchIndex(sample_term, inverted_index)
    print(f"Documents containing '{sample_term}': {list(matching_docs)} ...")


Inverted index built.
Documents containing 'inform': [2, 3, 4, 6, 12, 15, 17, 18, 23, 27, 28, 29, 30, 32, 33, 34, 37, 41, 47, 49, 53, 54, 57, 59, 60, 62, 63, 64, 66, 67, 72, 74, 78, 79, 80, 81, 85, 90, 95, 96, 97, 98, 107, 109, 112, 114, 119, 120, 121, 122, 123, 125, 126, 127, 128, 129, 130, 131, 132, 133, 135, 136, 137, 138, 140, 141, 142, 145, 147, 150, 151, 152, 155, 156, 158, 159, 160, 161, 163, 164, 166, 169, 173, 174, 175, 176, 177, 178, 179, 180, 184, 190, 199, 202, 206, 213, 216, 218, 220, 224, 225, 228, 231, 236, 241, 243, 244, 245, 248, 252, 254, 257, 258, 259, 260, 267, 270, 274, 293, 309, 310, 311, 314, 319, 320, 321, 323, 324, 327, 328, 330, 334, 336, 338, 339, 340, 341, 344, 345, 346, 347, 348, 352, 360, 362, 363, 367, 371, 372, 373, 375, 376, 378, 381, 386, 388, 389, 392, 398, 400, 403, 406, 408, 411, 412, 419, 420, 421, 425, 426, 429, 433, 434, 439, 440, 442, 444, 445, 446, 449, 451, 452, 453, 454, 456, 458, 459, 460, 461, 462, 463, 466, 469, 471, 473, 474, 475, 476, 47

# Boolean Search Implementation

Τώρα χτίζουμε τη λογική για Boolean Expressions τύπου information AND system, information OR system, information NOT system κ.λπ. Κάνουμε parsing της έκφρασης και χρησιμοποιούμε σύνολα για AND/OR/NOT.


In [45]:
# ============================================================
# 5) Boolean Search
# ============================================================

def evaluate_expression(expression, articles):
    """
    Υλοποιεί τη Boolean αναζήτηση που υποστηρίζει AND, OR, NOT.
    Κάνει tokenize στην έκφραση, και χρησιμοποιεί μια στοίβα για
    αξιολόγηση (βασική postfix/stack λογική).
    """
    stack = []
    tokens = expression.split()
    for token in tokens:
        token_up = token.upper()
        if token_up in {"AND", "OR", "NOT"}:
            stack.append(token_up)
        else:
            matching = searchIndex(token, articles)
            stack.append(matching)
        # Όταν έχουμε σχήμα [SET, 'AND/OR/NOT', SET] το αξιολογούμε
        while len(stack) >= 3 and isinstance(stack[-1], set) and isinstance(stack[-3], set):
            right = stack.pop()
            operator = stack.pop()
            left = stack.pop()

            if operator == "AND":
                stack.append(left & right)
            elif operator == "OR":
                stack.append(left | right)
            elif operator == "NOT":
                stack.append(left - right)

    if len(stack) == 1 and isinstance(stack[0], set):
        return stack[0]
    else:
        return set()

def boolean_search(query_text, articles):
    """
    Επιστρέφει μια λίστα από doc_ids που ταιριάζουν στη Boolean έκφραση.
    Σε περίπτωση που αποτύχει η parsing (λχ λάθος syntax), πέφτει στο "fallback OR".
    """
    query_text = query_text.strip()
    try:
        result_set = evaluate_expression(query_text, articles)
        if not result_set:
            raise ValueError("Empty result from Boolean")
        return list(result_set), []
    except:
        # fallback: αν δεν δουλέψει η έκφραση, κάνουμε OR σε όλα τα tokens
        processed = process_query(query_text)
        results = set()
        for t in processed:
            results.update(searchIndex(t, articles))
        return list(results), []

# -----------------------------------------------------------
# Δοκιμή Boolean Search
# -----------------------------------------------------------

test_query = "information"
results_boolean = boolean_search(test_query, inverted_index)
print(f"Results for query '{test_query}': {results_boolean} ...")

test_query = "system"
results_boolean = boolean_search(test_query, inverted_index)
print(f"Results for query '{test_query}': {results_boolean} ...")

test_query = "information AND system"
results_boolean = boolean_search(test_query, inverted_index)
print(f"Results for query '{test_query}': {results_boolean} ...")

Results for query 'information': ([2, 3, 4, 6, 12, 15, 17, 18, 23, 27, 28, 29, 30, 32, 33, 34, 37, 41, 47, 49, 53, 54, 57, 59, 60, 62, 63, 64, 66, 67, 72, 74, 78, 79, 80, 81, 85, 90, 95, 96, 97, 98, 107, 109, 112, 114, 119, 120, 121, 122, 123, 125, 126, 127, 128, 129, 130, 131, 132, 133, 135, 136, 137, 138, 140, 141, 142, 145, 147, 150, 151, 152, 155, 156, 158, 159, 160, 161, 163, 164, 166, 169, 173, 174, 175, 176, 177, 178, 179, 180, 184, 190, 199, 202, 206, 213, 216, 218, 220, 224, 225, 228, 231, 236, 241, 243, 244, 245, 248, 252, 254, 257, 258, 259, 260, 267, 270, 274, 293, 309, 310, 311, 314, 319, 320, 321, 323, 324, 327, 328, 330, 334, 336, 338, 339, 340, 341, 344, 345, 346, 347, 348, 352, 360, 362, 363, 367, 371, 372, 373, 375, 376, 378, 381, 386, 388, 389, 392, 398, 400, 403, 406, 408, 411, 412, 419, 420, 421, 425, 426, 429, 433, 434, 439, 440, 442, 444, 445, 446, 449, 451, 452, 453, 454, 456, 458, 459, 460, 461, 462, 463, 466, 469, 471, 473, 474, 475, 476, 477, 478, 480, 481, 4

# TF-IDF Implementation (Dot Product)
Τώρα θα υλοποιήσουμε μια απλή TF-IDF προσέγγιση, χρησιμοποιώντας TfidfVectorizer από το scikit-learn. Υπολογίζουμε τον πίνακα TF-IDF για όλα τα άρθρα και κάνουμε dot-product με το query vector.


In [ ]:
# ============================================================
# 6) TF-IDF (dot product)
# ============================================================

def rank_tfidf(query_tokens, articles, inverted_index):
    # 1) Βρίσκουμε τα σχετικά έγγραφα (doc_ids) από το inverted index
    relevant_doc_ids = set()
    for token in query_tokens:
        relevant_doc_ids.update(inverted_index.get(token, []))

    # 2) Φτιάχνουμε ένα υποσύνολο των άρθρων
    relevant_docs = [a for a in articles if a['id'] in relevant_doc_ids]

    # 3) Δημιουργούμε το document-term matrix (TF-IDF) για αυτά τα docs
    corpus = [" ".join(a['tokens']) for a in relevant_docs]
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(corpus)

    # 4) Μετατρέπουμε το query σε vector μέσω του ίδιου vectorizer
    query_vec = vectorizer.transform([" ".join(query_tokens)])

    # 5) Υπολογίζουμε το dot product (tfidf_matrix * query_vec)
    scores = (tfidf_matrix @ query_vec.T).toarray().flatten()
    # Το πολλαπλασιάζουμε επί 100 για κλίμακα 0-100
    scores *= 100

    # 6) Ταξινομούμε φθίνουσα
    ranked_indices = np.argsort(-scores)
    # Εξάγουμε doc_ids και scores
    ranked_docs = [relevant_docs[i]['id'] for i in ranked_indices]
    ranked_scores = [scores[i] for i in ranked_indices]
    return ranked_docs, ranked_scores

# -----------------------------------------------------------
# Δοκιμή
# -----------------------------------------------------------
query_text = "information retrieval systems"
test_query_tokens = process_query(query_text)
ranked_indices_tfidf, scores_tfidf = rank_tfidf(test_query_tokens, articles, inverted_index) # Added inverted_index as an argument

print(f"Top 5 document indices using TF-IDF for query: {query_text}")
for i in ranked_indices_tfidf[:5]:
    # Βρίσκουμε το index του άρθρου που αντιστοιχεί στο doc ID
    article_index = next((index for index, article in enumerate(articles) if article['id'] == i), None)
    if article_index is not None:  # Αν βρέθηκε το άρθρο
        print(f"Doc index = {i}, Score = {scores_tfidf[ranked_indices_tfidf.index(i)]:.2f}, ID = {articles[article_index]['id']}")
    else:
        print(f"Article with ID {i} not found in the articles list.")

Top 5 document indices using TF-IDF for query: information retrieval systems
Doc index = 565, Score = 47.77, ID = 565
Doc index = 1136, Score = 46.01, ID = 1136
Doc index = 459, Score = 34.52, ID = 459
Doc index = 458, Score = 33.15, ID = 458
Doc index = 538, Score = 32.95, ID = 538


# BM25 Implementation
Η BM25 είναι μια πιο εξελιγμένη μέθοδος που στηρίζεται σε TF, IDF και μήκος εγγράφου. Παρακάτω την υλοποιούμε σε δύο βήματα:

1. calc_idf για BM25
2. BM25 συνάρτηση που υπολογίζει το score κάθε εγγράφου

In [ ]:
# ---------------------------------------------------------
# BM25
# ---------------------------------------------------------
"""
Η μέθοδος BM25 είναι ένας αλγόριθμος ranking που βασίζεται σε TF, IDF
και σε δύο παραμέτρους (k1 και b). Υπολογίζει για κάθε όρο q και έγγραφο d:
  score(d) += IDF(q) * ( (TF(q,d)*(k1+1)) / (TF(q,d) + k1*(1 - b + b*(|d|/avg|d|))) )
"""
def calc_idf(articles):
    # Υπολογισμός idf για BM25: log( (N - df + 0.5)/(df + 0.5) + 1 )
    N = len(articles)
    term_doc_count = defaultdict(int)
    for article in articles:
        unique_tokens = set(article['tokens'])
        for token in unique_tokens:
            term_doc_count[token] += 1
    idf = {}
    for token, doc_count in term_doc_count.items():
        idf[token] = math.log((N - doc_count + 0.5) / (doc_count + 0.5) + 1)
    return idf

def rank_bm25(query_tokens, articles, idf, inverted_index):
    """
    Βρίσκουμε πάλι μόνο τα σχετικά έγγραφα από το inverted index, κι
    έπειτα υπολογίζουμε το BM25 score για το καθένα βάσει TF, IDF, k1, b.
    """
    k1 = 1.5
    b = 0.75
    N = len(articles)
    avg_len = sum(len(a['tokens']) for a in articles) / N

    # 1) σχετικά doc_ids
    relevant_doc_ids = set()
    for token in query_tokens:
        relevant_doc_ids.update(inverted_index.get(token, []))

    # 2) Φτιάχνουμε τη λίστα των relevant_docs
    relevant_docs = [a for a in articles if a['id'] in relevant_doc_ids]

    # 3) Υπολογισμός BM25 score
    scores = []
    for a in relevant_docs:
        freq = defaultdict(int)
        for t in a['tokens']:
            freq[t] += 1

        score = 0
        for q in query_tokens:
            if q in idf:
                tf = freq[q]
                numerator = tf * (k1 + 1)
                denominator = tf + k1 * (1 - b + b * (len(a['tokens']) / avg_len))
                score += idf[q] * (numerator / denominator)

        scores.append(score)

    scores = np.array(scores)
    # Ταξινόμηση
    ranked_indices = np.argsort(-scores)
    ranked_docs = [relevant_docs[i]['id'] for i in ranked_indices]
    ranked_scores = [scores[i] for i in ranked_indices]
    return ranked_docs, ranked_scores

# -----------------------------------------------------------
# Υπολογίζουμε τα idf_values μια φορά
# -----------------------------------------------------------
idf_values = calc_idf(articles)

# -----------------------------------------------------------
# Δοκιμή BM25
# -----------------------------------------------------------
ranked_indices_bm25, scores_bm25 = rank_bm25(test_query_tokens, articles, idf_values, inverted_index)
print(f"Top 5 document indices using BM25 for query: {query_text}")
for doc_index in ranked_indices_bm25[:5]:
    article_index = next((index for index, article in enumerate(articles) if article['id'] == doc_index), None)

    if article_index is not None:
        print(f"Doc index = {doc_index}, Score = {scores_bm25[ranked_indices_bm25.index(doc_index)]:.2f}, ID = {articles[article_index]['id']}")
    else:
        print(f"Article with ID {doc_index} not found in the articles list.")


Top 5 document indices using BM25 for query: information retrieval systems
Doc index = 1136, Score = 7.12, ID = 1136
Doc index = 565, Score = 7.09, ID = 565
Doc index = 459, Score = 6.52, ID = 459
Doc index = 445, Score = 6.52, ID = 445
Doc index = 611, Score = 6.38, ID = 611


# Vector Space Model
Στο μοντέλο διανυσματικού χώρου (VSM), κάθε έγγραφο ή ερώτημα είναι ένα διάνυσμα Ν-διαστάσεων, όπου Ν είναι ο αριθμός των διαφορετικών όρων σε όλα τα έγγραφα και τα ερωτήματα.Ο i-οστός δείκτης ενός διανύσματος περιέχει τη βαθμολογία του i-οστού όρου για το συγκεκριμένο διάνυσμα. Χρησιμοποιούνται TF-IDF και Cosine Similarity.

In [50]:
# ============================================================
# TF-IDF + Cosine Similarity
# ============================================================
def rank_vsm(query_tokens, articles, inverted_index):
    """
    Υλοποιεί αναζήτηση βασισμένη σε TF-IDF και Cosine Similarity,
    μόνο για έγγραφα που περιέχουν τουλάχιστον έναν από τους όρους του query.

    Βήματα:
    1) Εύρεση συναφών εγγράφων (relevant_docs) μέσω του inverted_index.
    2) Δημιουργία TF-IDF matrix μόνο για αυτά τα έγγραφα.
    3) Δημιουργία vector για το query.
    4) Υπολογισμός cosine_similarity.
    5) Κανονικοποίηση των scores σε 0..100.
    6) Επιστροφή ταξινομημένης λίστας doc_ids και των αντίστοιχων scores.
    """

    # 1) Εύρεση relevant_doc_ids
    relevant_doc_ids = set()
    for token in query_tokens:
        relevant_doc_ids.update(inverted_index.get(token, []))

    # 2) Φιλτράρουμε τα articles για να κρατήσουμε μόνο τα σχετικά
    relevant_docs = [a for a in articles if a['id'] in relevant_doc_ids]

    # 3) Δημιουργούμε ένα corpus για το TfidfVectorizer
    corpus = [" ".join(a['tokens']) for a in relevant_docs]

    # Check if the corpus is empty after preprocessing
    if not any(corpus):
        print("Warning: Corpus is empty after preprocessing. Returning empty results.")
        return [], [] # Return empty lists to indicate no results

    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(corpus)

    # 4) Φτιάχνουμε το query vector
    query_vec = vectorizer.transform([" ".join(query_tokens)])

    # 5) Υπολογίζουμε cosine similarity
    cos_sims = cosine_similarity(query_vec, tfidf_matrix).flatten()

    # 6) Κλίμακα 0..100
    cos_sims *= 100

    # 7) Ταξινόμηση φθίνουσα
    ranked_indices = np.argsort(-cos_sims)
    ranked_docs = [relevant_docs[i]['id'] for i in ranked_indices]
    ranked_scores = [cos_sims[i] for i in ranked_indices]

    return ranked_docs, ranked_scores

test_query_tokens = ["quick", "history"]
docs_vsm, scores_vsm = rank_vsm(test_query_tokens, articles, inverted_index)
print("Top 5 Docs (TF-IDF + Cosine):", docs_vsm[:5])
print("Scores:", scores_vsm)


Top 5 Docs (TF-IDF + Cosine): [174, 928]
Scores: [6.561619460946319, 3.592972573809947]


# Συνάρτηση Ranking
Σε αυτή τη φάση, ενώνουμε τις διάφορες μεθόδους (Boolean, TF-IDF, BM25, TF-IDF+Cosine) σε μία κεντρική συνάρτηση ranking

In [52]:
# ============================================================
# Συνάρτηση Ranking (με 4 methods)
# ============================================================

def ranking(articles, query_text, method, inverted_index):
    """
    Ανάλογα με το 'method':
      '1' -> Boolean
      '2' -> TF-IDF (dot product)
      '3' -> BM25
      '4' -> TF-IDF + Cosine
    """
    # Προεπεξεργασία του query
    processed_query = process_query(query_text)

    if method == '1':
        # Boolean
        docs, _ = boolean_search(query_text, inverted_index)
        return docs, []

    elif method == '2':
        # TF-IDF (dot product)
        # π.χ. θα χρησιμοποιήσουμε rank_tfidf_vectorizer
        ranked_indices, sc = rank_tfidf(processed_query, articles, inverted_index)
        # ranked_indices είναι λίστα από doc_ids, όχι indices
        #doc_ids = [articles[i]['id'] for i in ranked_indices] # This line is removed
        doc_ids = ranked_indices # ranked_indices already contain doc_ids
        scores_ordered = [sc[i] for i in range(len(ranked_indices))] # Using a range of indices
        return doc_ids, scores_ordered

    elif method == '3':
        # BM25
        idf_vals = calc_idf(articles)
        doc_ids, scores_ordered = rank_bm25(processed_query, articles, idf_vals, inverted_index) # Added inverted_index
        return doc_ids, scores_ordered

    elif method == '4':
        # TF-IDF + Cosine
        doc_ids, scores_ordered = rank_vsm(processed_query, articles, inverted_index) # Added inverted_index, removed relevant_only
        return doc_ids, scores_ordered

    else:
        print("Μη έγκυρη μέθοδος.")
        return [], []

# -----------------------------------------------------------
# Δοκιμή της ranking() συνάρτησης
# -----------------------------------------------------------
test_query2 = "information science"
for m in ['1','2','3','4']:
    doc_list, scores = ranking(articles, test_query2, m, inverted_index)
    print(f"Method={m}, first 5 results:")
    for i in range(min(5, len(doc_list))):
        did = doc_list[i]
        sc  = scores[i] if scores else 0
        print(f"  DocID={did}, Score={sc:.2f}")


Method=1, first 5 results:
  DocID=2, Score=0.00
  DocID=3, Score=0.00
  DocID=4, Score=0.00
  DocID=6, Score=0.00
  DocID=12, Score=0.00
Method=2, first 5 results:
  DocID=469, Score=43.51
  DocID=456, Score=38.03
  DocID=1030, Score=35.77
  DocID=599, Score=34.54
  DocID=85, Score=34.50
Method=3, first 5 results:
  DocID=469, Score=5.45
  DocID=599, Score=5.17
  DocID=85, Score=5.03
  DocID=456, Score=5.00
  DocID=137, Score=5.00
Method=4, first 5 results:
  DocID=469, Score=43.51
  DocID=456, Score=38.03
  DocID=1030, Score=35.77
  DocID=599, Score=34.54
  DocID=85, Score=34.50


# main_loop (Interactive ή One-shot)
Εδώ έχουμε τη συνάρτηση που, αν use='1', καλείται σε one-shot mode (π.χ. για αυτόματες κλήσεις όπως ground truth). Αλλιώς (default) μπαίνει σε interactive CLI mode.



In [53]:
# ============================================================
# main_loop (διαδραστικό ή one-shot)
# ============================================================

def main_loop(articles, title_mapping, query=None, use='0', method=None):
    global _inverted_index
    if use == '1':
        # one-shot λειτουργία: δεν κάνουμε interactive
        _inverted_index = buildInvertedIndex(articles)
        doc_ids, scores = ranking(articles, query, method, _inverted_index)
        return doc_ids, scores
    else:
        # interactive
        while True:
            print("\nMenu:")
            print("1) Search")
            print("2) Exit")
            choice = input("Choice: ").strip()
            if choice == '1':
                user_query = input("Enter your query: ")
                print("Methods:\n1) Boolean\n2) TF-IDF\n3) BM25\n4) TF-IDF + Cosine")
                user_method = input("Select (1..4): ").strip()
                # Φτιάχνουμε/ανανέουμε το inverted_index πριν το ranking
                _inverted_index = buildInvertedIndex(articles)
                docs, scores = ranking(articles, user_query, user_method, _inverted_index)

                top_k = min(10, len(docs))
                for i in range(top_k):
                    did = docs[i]
                    sc = scores[i] if scores else 0
                    title = title_mapping.get(did, "No Title")
                    print(f"{i+1}. DocID={did}, Score={sc:.2f}, Title={title}")
            elif choice == '2':
                print("Exiting interactive mode.")
                break
        return [], []

# -----------------------------------------------------------
# Παράδειγμα: Κλήση main_loop σε One-shot mode
# -----------------------------------------------------------
print("\n=== One-shot example ===")
test_q = "information retrieval system"
doc_ids_test, sc_test = main_loop(articles, title_mapping, query=test_q, use='1', method='2')
print(f"One-shot, method=2, found {len(doc_ids_test)} docs. Top 5 doc IDs:")
print(doc_ids_test[:5])



=== One-shot example ===
One-shot, method=2, found 846 docs. Top 5 doc IDs:
[565, 1136, 459, 458, 538]


# Ground Truth Creation & Parse Relevance
Δημιουργία Ground Truth (CISI.REL)


In [54]:
# ============================================================
# Ground Truth creation (CISI.REL)
# ============================================================

def ground_truth(articles, title_mapping, queries, method_for_gt='3'):
    """
    Δημιουργεί/ενημερώνει το αρχείο CISI.REL με μορφή:
      query_id doc_id relevance score
    όπου η relevance καθορίζεται από thresholds στο score.
    """
    data = []
    for qid, qtext in queries.items():
        # Καλούμε main_loop σε one-shot mode με τη ζητούμενη μέθοδο
        ranked_docs, scores = main_loop(articles, title_mapping, qtext, use='1', method=method_for_gt)
        query_data = []
        for doc_id, sc in zip(ranked_docs, scores):
            # Παράδειγμα thresholding
            if sc < 10:
                relevance = 0
            elif 10 <= sc < 30:
                relevance = 1
            else:
                relevance = 2
            query_data.append((qid, doc_id, relevance, sc))
        data.extend(query_data)

    # Γράφουμε σε CISI.REL
    with open("./CISI.REL", "w") as f:
        for row in data:
            f.write("{:5d} {:5d} {:1d} {:10.4f}\n".format(row[0], row[1], row[2], row[3]))
    print("CISI.REL updated successfully.")

# -----------------------------------------------------------
# Παράδειγμα: Δημιουργούμε ground truth με μέθοδο BM25
# -----------------------------------------------------------
print("Creating ground truth (CISI.REL) with method=3 (BM25) ...")
ground_truth(articles, title_mapping, queries, method_for_gt='3')


Creating ground truth (CISI.REL) with method=3 (BM25) ...
CISI.REL updated successfully.


Parse Relevance (διαβάζει CISI.REL)

In [55]:
# ============================================================
# Parse Relevance
# ============================================================

def parse_relevance(file_path):
    """
    Διαβάζει το CISI.REL και φτιάχνει ένα λεξικό:
      { query_id: [doc_ids με rel>0] }
    """
    relevance_dict = {}
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            if len(parts) >= 3:
                qid = int(parts[0])
                did = int(parts[1])
                rel = int(parts[2])
                if qid not in relevance_dict:
                    relevance_dict[qid] = []
                if rel > 0:
                    relevance_dict[qid].append(did)
    return relevance_dict

# -----------------------------------------------------------
# Δοκιμή
# -----------------------------------------------------------
rel_dict = parse_relevance("./CISI.REL")
print(f"Parsed relevance from CISI.REL: found {len(rel_dict)} queries with rel>0 info.")


Parsed relevance from CISI.REL: found 57 queries with rel>0 info.


# Evaluate Search Engine (Precision, Recall, F1)
Τέλος, δείχνουμε πώς να κάνουμε evaluation χρησιμοποιώντας το ground truth που δημιουργήσαμε:



In [56]:
# ============================================================
# Evaluate Search Engine
# ============================================================

def eval_search_engine(queries, ground_truth_dict, articles, title_mapping):
    """
    Κάνει Evaluation με precision, recall, F1.
    Ζητάει μέθοδο (1..4), μετά για κάθε query:
      - τρέχει main_loop (one-shot)
      - υπολογίζει tp, fp, fn
    """
    print("\nSelect Ranking Method for Evaluation:")
    print("1) Boolean Search")
    print("2) TF-IDF (dot product)")
    print("3) Okapi BM25")
    print("4) TF-IDF + Cosine Similarity")
    method_choice = input("Enter your choice (1/2/3/4): ").strip()

    precision_scores = []
    recall_scores = []
    f1_scores = []

    for qid, qtext in queries.items():
        print(f"\nEvaluating Query ID {qid}: {qtext}")
        doc_ids, _scores = main_loop(articles, title_mapping, qtext, use='1', method=method_choice)
        retrieved_docs = set(doc_ids)
        relevant_docs = set(ground_truth_dict.get(qid, []))

        tp = len(retrieved_docs & relevant_docs)
        fp = len(retrieved_docs - relevant_docs)
        fn = len(relevant_docs - retrieved_docs)

        precision = tp/(tp+fp) if (tp+fp) > 0 else 0
        recall = tp/(tp+fn) if (tp+fn) > 0 else 0
        f1 = 2*precision*recall/(precision+recall) if (precision+recall) > 0 else 0

        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)

        print(f"Precision={precision:.2f}, Recall={recall:.2f}, F1={f1:.2f}")

    avg_p = sum(precision_scores)/len(precision_scores) if precision_scores else 0
    avg_r = sum(recall_scores)/len(recall_scores) if recall_scores else 0
    avg_f1 = sum(f1_scores)/len(f1_scores) if f1_scores else 0

    print("\nOverall Performance:")
    print(f"Avg Precision: {avg_p:.2f}")
    print(f"Avg Recall:    {avg_r:.2f}")
    print(f"Avg F1-Score:  {avg_f1:.2f}")

# -----------------------------------------------------------
# Δοκιμή evaluation
# -----------------------------------------------------------
print("\n=== Evaluate Search Engine ===")
ground_truth_dict = parse_relevance("./CISI.REL")
eval_search_engine(queries, ground_truth_dict, articles, title_mapping)



=== Evaluate Search Engine ===

Select Ranking Method for Evaluation:
1) Boolean Search
2) TF-IDF (dot product)
3) Okapi BM25
4) TF-IDF + Cosine Similarity
Enter your choice (1/2/3/4): 3

Evaluating Query ID 1: What problems and concerns are there in making up descriptive titles? What difficulties are involved in automatically retrieving articles from approximate titles? What is the usual relevance of the content of articles to their titles?
Precision=0.12, Recall=1.00, F1=0.21

Evaluating Query ID 2: How can actually pertinent data, as opposed to references or entire articles themselves, be retrieved automatically in response to information requests?
Precision=0.03, Recall=1.00, F1=0.07

Evaluating Query ID 3: What is information science?  Give definitions where possible.
Precision=0.00, Recall=1.00, F1=0.01

Evaluating Query ID 4: Image recognition and any other methods of automatically transforming printed text into computer-ready form.
Precision=0.02, Recall=1.00, F1=0.04

Evaluat